# Reporting Applied Prediction Probabilities - Replication Data- (for STREAMLINE)
The pipeline outputs pickled objects with all the metric results during the initial testing evaluation of trained models as well as following application of trained models to additional hold out replication data (i.e. apply data).

This notebook grabs these prediction probabilities on the replication data and reports them as .csv files for each algorithm and CV partition pair (i.e for each of the CV trained models).  Unlike the initial testing evaluation, here prediction probabilities for all instances in the replication data are reported rather than just for those in 'testing' holdout data subsets.

In these files is the instance's true outcome value, the unique instance ID, and the predicted probability of the instance being case/code 1. 

This code is set up to run on a specific pair of an original dataset and a paired replication dataset one at a time.
 

## Import Packages

In [22]:
import os
import pandas as pd
import pickle
import numpy as np
from statistics import mean
from scipy import interp,stats
import warnings
warnings.filterwarnings('ignore')

# Jupyter Notebook Hack: This code ensures that the results of multiple commands within a given cell are all displayed, rather than just the last. 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Set Run Parameters

In [23]:
experiment_path = "../demo/demo"
dataname = 'hcc-data_example_custom' #name of target dataset folder in experiment output folder from pipeline
rep_data_path ="../data/DemoRepData/hcc-data_example_custom_rep.csv"#path to replication dataset file (needed to grab instance labels and true class values)
algorithms = [] #use empty list if user wishes re-evaluate all modeling algorithms that were run in pipeline.

## Load Other Necessary Parameters

In [24]:
# Unpickle metadata from previous phase
file = open(experiment_path+'/'+"metadata.pickle", 'rb')
metadata = pickle.load(file)
file.close()
# Load variables specified earlier in the pipeline from metadata
class_label = metadata['Class Label']
instance_label = metadata['Instance Label']
cv_partitions = int(metadata['CV Partitions'])

# Unpickle algorithm information from previous phase
file = open(experiment_path+'/'+"algInfo.pickle", 'rb')
algInfo = pickle.load(file)
file.close()
algorithms = []
abbrev = {}
colors = {}
for key in algInfo:
    if algInfo[key][0]: # If that algorithm was used
        algorithms.append(key)
        abbrev[key] = (algInfo[key][1])
        colors[key] = (algInfo[key][2])
        
print("Algorithms Ran: " + str(algorithms))

Algorithms Ran: ['Decision Tree', 'Logistic Regression', 'Naive Bayes']


## Extract and Report Case (i.e. class 1) Prediction Probabilities For all instances in replication dataset applied to all CV models.

In [25]:
full_path = experiment_path+'/'+dataname
apply_name = rep_data_path.split('/')[-1].split('.')[0]
new_full_path = full_path+'/applymodel/'+apply_name
        
#Make folder in experiment folder/datafolder to store all prediction probabilities per algorithm/CV combination
if not os.path.exists(new_full_path+'/model_evaluation/prediction_probas'):
    os.mkdir(new_full_path+'/model_evaluation/prediction_probas')

for algorithm in algorithms: #loop through algorithms
    print(algorithm)

    for cvCount in range(0,cv_partitions): #loop through cv's
        print(cvCount)
        #Load pickled metric file for given algorithm and cv
        result_file = new_full_path+'/model_evaluation/pickled_metrics/'+abbrev[algorithm]+"_CV_"+str(cvCount)+"_metrics.pickle"
        file = open(result_file, 'rb')
        results = pickle.load(file)
        file.close()

        #Load target replication dataset (From which we will get the instancelabel values and class outcome values.)
        rep_data = pd.read_csv(rep_data_path)
        probas_summary = rep_data[[class_label,instance_label]]

        #Separate pickled results
        probas_ = results[9]
        try:
            print(probas_[:,1])
            probas_summary['1_prob']=probas_[:,1]
            file_name = new_full_path+'/model_evaluation/prediction_probas/'+algorithm+'_CV_'+str(cvCount)+'_case_probas.csv'
            probas_summary.to_csv(file_name, index=False)
        except:
            print("prediction_probas not available")

Decision Tree
0
[0.         0.375      0.375      0.5        0.25       0.66666667
 0.375      0.5        0.375      0.08333333 0.08333333 0.28571429
 0.28571429 0.28571429 0.55555556 0.08333333 0.375      0.55555556
 0.375      0.375      0.375      0.         1.         0.63157895
 0.         0.63157895 0.66666667 0.         0.63157895 0.
 0.66666667 0.         0.55555556 0.08333333 0.08333333 0.08333333
 0.375      0.375      0.08333333 0.375      0.08333333 0.08333333
 0.375      0.         0.375      1.         0.66666667 1.
 0.55555556 0.08333333 0.375      0.375      0.28571429 0.
 1.         1.         0.63157895 0.08333333 0.28571429 0.55555556
 0.63157895 0.         0.5        0.28571429 0.28571429 0.375
 0.5        0.63157895 0.5        0.375      0.375      0.63157895
 0.63157895 1.         0.08333333 0.375      0.63157895 0.
 1.         0.55555556 0.25       0.         0.375      0.63157895
 0.55555556 0.375      1.         0.         0.5        0.
 0.08333333 0.08333333 0